In [181]:
import pandas as pd
from gurobipy import Model, GRB
from gurobipy import Model, GRB, quicksum



In [182]:
costs_df = pd.read_csv('/Users/aimaldastagirzada/Downloads/costs.csv')

In [183]:
N = len(costs_df)  # Number of potential sites
Fixed_costs = costs_df['Fixed'].values

In [184]:
m = Model('HealthLink_Supply_Chain_Diversification')


In [185]:
y = m.addVars(N, vtype=GRB.BINARY, name="y")  # Whether to open a warehouse at site i


In [186]:
units = m.addVars(N, lb=0, ub=375000, name="units")

In [187]:
# Objective Function
m.setObjective(quicksum(Fixed_costs[i] * y[i] for i in range(N)), GRB.MINIMIZE)


In [188]:
for i in range(N):
    m.addConstr(units[i] >= 175000 * y[i], f"Min_units_{i+1}")
    m.addConstr(units[i] <= 375000 * y[i], f"Max_units_{i+1}")

In [189]:
m.addConstr(quicksum(y[i] for i in range(5, 16)) >= 4, "Min_Locations_6_16")

# Constraint: No more than 6 locations chosen amongst even-numbered sites
m.addConstr(quicksum(y[i] for i in range(1, N, 2)) <= 6, "Max_Even_Sites")

# Constraint: If location 1 or 2 is chosen, then sites 5, 6, and 7 cannot be chosen
m.addConstr(y[0] + y[1] <= 1, "Exclusion1")
m.addConstr(y[4] + y[5] + y[6] <= 2*(1-y[0]), "Exclusion2_5_6_7_if_1")
m.addConstr(y[4] + y[5] + y[6] <= 2*(1-y[1]), "Exclusion2_5_6_7_if_2")

# Constraint: If any location from 19-22 is chosen, locations 24, 26, and 27 cannot be chosen
for i in range(18, 22):
    m.addConstr(y[i] + y[23] + y[25] + y[26] <= 1, f"Mutual_Exclusion_{i+1}_with_24_26_27")

# Constraint: If any location from 1-5 is chosen, at least one odd site from 21-27 must be chosen
m.addConstr(quicksum(y[i] for i in range(5)) <= quicksum(y[j] for j in range(20, 27, 2)), "Conditional_Selection_1_5_with_21_27")

# Constraint: The number of locations chosen in 1-14 must equal the number of locations chosen in 15-27
m.addConstr(quicksum(y[i] for i in range(14)) == quicksum(y[i] for i in range(14, 27)), "Equal_Selection_1_14_and_15_27")

# Constraint: The sum of all units at sites 1-9 and 19-27 must be equal
m.addConstr(quicksum(units[i] for i in range(9)) == quicksum(units[i] for i in range(18, 27)), "Equal_Units_1_9_19_27")


<gurobi.Constr *Awaiting Model Update*>

In [190]:
# Solve the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 66 rows, 54 columns and 212 nonzeros
Model fingerprint: 0x63c82321
Variable types: 27 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  Objective range  [1e+06, 3e+06]
  Bounds range     [1e+00, 4e+05]
  RHS range        [1e+00, 6e+00]
Presolve removed 19 rows and 9 columns
Presolve time: 0.01s
Presolved: 47 rows, 45 columns, 174 nonzeros
Variable types: 18 continuous, 27 integer (27 binary)
Found heuristic solution: objective 9980000.0000

Root relaxation: objective 6.440000e+06, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6440000.0000 6440000.00  0.00%     

In [191]:
# Output the solution
if m.status == GRB.OPTIMAL:
    print(f"Optimal total cost: {m.objVal}")
    for i in range(N):
        if y[i].X > 0.5:  # If a warehouse is established at site i
            print(f"Warehouse at site {i+1} is selected with {units[i].X} units.")

Optimal total cost: 6440000.0
Warehouse at site 11 is selected with 375000.0 units.
Warehouse at site 13 is selected with 375000.0 units.
Warehouse at site 15 is selected with 375000.0 units.
Warehouse at site 16 is selected with 375000.0 units.
